In [ ]:
import requests 
from bs4 import BeautifulSoup

from dataclasses import dataclass

@dataclass
class Stock:
    ticker: str
    exchange: str
    price: float=0
    currency: str='USD'
    usd_price: float=0

    def __post_init__(self):                                    # executes after initialization of class & its properties
        price_info=get_price_info(self.ticker,self.exchange)    # have to get hold of values to instance of stock being created, self points to Stock type Object
        if price_info['ticker']==self.ticker:                   # to make sure all info is correct
            self.price=price_info['price']
            self.currency=price_info['currency']
            self.usd_price=price_info['usd_price']


def get_fx_to_usd(currency):
    fx_url=f'https://www.google.com/finance/quote/{currency}-USD' 
    response=requests.get(fx_url)                                    
    soup=BeautifulSoup(response.content,"html.parser")            
    fx_rate=soup.find('div',attrs={'data-last-price':True})     
    fx=float(fx_rate['data-last-price'])
    return fx


def get_price_info(ticker,exchange):                                    
    url=f'https://www.google.com/finance/quote/{ticker}:{exchange}'     
    response=requests.get(url)                                       
    soup=BeautifulSoup(response.content,"html.parser")               
    price_div=soup.find('div',attrs={'data-last-price':True})        
    price=float(price_div['data-last-price'])                        
    currency=price_div['data-currency-code']                         
    
    usd_price=price
    if(currency!='USD'):                                             
        fx=get_fx_to_usd(currency)
        usd_price=round(price*fx,2)
    
    return {
        'ticker':ticker,
        'exchange':exchange,
        'price':price,
        'currency':currency,
        'usd_price':usd_price
    }